In this blog post I'll explain how I created this blog, using Quarto and GitHub. In step 4 I'll show how to setup GitHub Actions, this has advantages over the other ways to publish our blog:

1. Our source code is also stored safely on GitHub
2. We can easily create blog entries anywhere we want, especially also from our deep learning server without having to install quarto. We can simply push to the remote branch and GitHub Actions will build the blog for us.

I'm working on a Macbook, and using VS Code for code editing. If you are on a Linux or Windows machine, be aware that things might be a bit different from what I describe here.

I am assuming you already have a GitHub account, that VS Code is installed and configured to run Python and Jupyter Notebooks.

## Step 1: install Quarto

First of all you need to install Quarto, go [here](https://quarto.org/docs/get-started/), download and install the software. You should do this on the machine that you want to use for writing your blog, in my case my Macbook laptop.

Once installed you will have access to the quarto Command Line Interface (CLI). To make sure everything works as expected, open a terminal and execute:

```{.bash filename="Terminal"}
quarto --help
```

This should render some outputs describing the different commands and options that are part of the Quarto CLI and shows that Quarto is installed successfully.


## Step 2: create a GitHub repo

To host our blog we will use GitHub Pages, which is a service to host a website from a GitHub repository. Based on the name you pick for your repository you will create a so-called project-website or your unique user-website. For any general repo named `my-awesome-repo`, the website will be hosted on `https://<github-username>.github.io/my-awesome-repo`. This is a project-websites and you can create as many as you like. 

To create your user-website, you have to name the repo exactly like this: `<github-username>.github.io`, the user-website will be hosted at `https://<github-username>.github.io`.

This is exactly what I want, so I create a new repo with the name: `lucasvw.github.io`.

I find it helpful to add a `.gitignore` file with a Python template, to which we can later add some more entries to facilitate storing the right files on GitHub. Also make sure that the repo is Public (and not set to Private). Additionally, I added a README file and choose the Apache2 License.

Next, I clone this repo to my machine by running:

```{.bash filename="Terminal"}
git clone git@github.com:lucasvw/lucasvw.github.io.git
```

## Step 3: add a Quarto project to the repo

Next, open VS Code and open the cloned repo. Then access the VS Code terminal and run:

```{.bash filename="Terminal"}
quarto create-project --type website:blog
```

This will add a number of files to our repo, which represent the basic structure of our blog. Most importantly:

- `posts`: here we will create our blog entries (one subfolder per blog entry)
- `_quarto.yml`: configuration file for our blog such as the theme, name, GitHub and Twitter links
- `about.qmd`: source code for the "about" page.
- `index.qmd`: source code for the landing page.

:::{.callout-note}
`.qmd` files are like markdown files, but with lots of additional functionality from Quarto. Go [here](https://www.markdownguide.org/basic-syntax/) for more information on Markdown syntax and [here](https://nbdev.fast.ai/tutorials/qmd_intro.html) for Quarto Markdown
:::

To see what we currently have, let's render our blog locally:

```{.bash filename="Terminal"}
quarto preview
```

Alternatively, we can install the Quarto extension in VS Code, which will show a `render` button in the top right corner on any opened `qmd` file.

To publish the current contents to GitHub pages, we can run:

```{.bash filename="Terminal"}
quarto publish gh-pages
```

When doing so, we get a message that we have to change the branch from which GitHub Pages builds the site. To do this, I go to [https://github.com/lucasvw/lucasvw.github.io/settings/pages](https://github.com/lucasvw/lucasvw.github.io/settings/pages) and select `gh-pages` instead of the `main` branch.

And voila, in a few moments our blog will be running live at [https://lucasvw.github.io/](https://lucasvw.github.io/)

## Step 4: Finalize set-up: GitHub Actions

When we run the `quarto publish gh-pages` command, Quarto processes our files and turns them into web readable files (HTML, JS, CSS etc). It stores these files in our `gh-pages` branch and pushes them to our remote GitHub repo. This is great, but it means that this doesn't store our source files.

To do so, let's first open our `.gitignore` file and make sure that it contains the following entries so that we don't check in any files we don't need.

```{.txt filename=".gitignore"}
# Quarto
/.quarto/
_site/

# Mac files
.DS_Store
```

Next, we can commit all the remaining files to Git and push them to our remote repo. If we ever lose access to our local machine, we can restore everything we need from GitHub.

However, now we have 2 things we need to do whenever we finish our work:

- store our source files on the main branch and push to GitHub
- run the publish command to update the blog

This is a bit annoying and it would be much better if we can just push to the main branch and GitHub would take care of building our website and updating it. This also allows us to create blog entries on any machine that has access to git, we don't need to have quarto installed. This is particularly practical if we want to write blog entries from our deep learning server. So let's use GitHub actions for this.

:::{.callout-note}
Before you continue make sure you have at least once run a `quarto publish gh-pages` command, this is necessary for the things below to work
:::

First we need to add the following snippet to `_quarto.yml`

```{.yml filename="_quarto.yml"}
execute:
  freeze: auto
```

This will make sure that GitHub actions doesn't execute any executable code, but will show the pre-rendered outputs it finds in the `_freeze` folder. 

Finally, create the file `.github/workflows/publish.yml` and populate it with the following code:

```{.yml filename=".github/workflows/publish.yml"}
on:
  workflow_dispatch:
  push:
    branches: main

name: Quarto Publish

jobs:
  build-deploy:
    runs-on: ubuntu-latest
    permissions:
      contents: write
    steps:
      - name: Check out repository
        uses: actions/checkout@v3

      - name: Set up Quarto
        uses: quarto-dev/quarto-actions/setup@v2

      - name: Render and Publish
        uses: quarto-dev/quarto-actions/publish@v2
        with:
          target: gh-pages
        env:
          GITHUB_TOKEN: ${{ secrets.GITHUB_TOKEN }}
```

Once we push these things to GitHub, we are good to go. Whenever we push anything to the `main` branch, this workflow will execute and take care of updating the `gh-pages` branch and updating the blog.